In [ ]:
# Get statewide count sums
grp_df=grp_df.groupby(by='state').sum().reset_index()


def plot_resp_by_state(columns, names, graphTitle):
    """Plots the proportion of patients who gave each response to a survey question
    args:
    columns = bracketed list of column names
    names = bracketed list of column names to-be
    graphTilte = string"""

    # create dict of current and to-be column names 
    name_dict={}

    for i, col in enumerate(columns):
        name_dict[col]=names[i]
    
    # add the following two columns to column list
    col_list=['state','number_of_completed_surveys']
    col_list.extend(columns)

    # create new df from grp_df and rename columns
    df=grp_df[col_list].rename(columns=name_dict)

    # change measure from count to proportion of total surveyed patients
    l=len(col_list)
    df.iloc[:,2:l]=df.values[:,2:l]/df.values[:,1,None]

    # restructure DataFrame
    df=df.melt(id_vars='state', value_vars=names, var_name='rating', value_name='proportion of patients')

    # create plot
    fig = go.Figure(data=[go.Box(x=df['rating'], y=df['proportion of patients'], boxpoints='all', jitter=0.5, width=.3, hoverinfo='text', hovertext=df['state'])])
    fig.update_layout(title_text=graphTitle, template="plotly_white")
    fig.update_xaxes(title_text="response")
    fig.update_yaxes(title_text="proportion of patients")
    fig.show()


# create a list of column lists, a list of name lists, and a list of graph ttiles for 4 measures

columns=[['patients_who_reported_that_staff_did_not_communicate_about_what_to_expect_during_and_after_the_procedure','patients_who_reported_that_staff_somewhat_communicated_about_what_to_expect_during_and_after_the_procedure', 'patients_who_reported_that_staff_definitely_communicated_about_what_to_expect_during_and_after_the_procedure'],
['patients_who_reported_no_they_would_not_recommend_the_facility_to_family_or_friends','patients_who_reported_probably_yes_they_would_recommend_the_facility_to_family_or_friends','patients_who_reported_yes_they_would_definitely_recommend_the_facility_to_family_or_friends'],
['patients_who_reported_that_staff_did_not_give_care_in_a_professional_way_or_the_facility_was_not_clean','patients_who_reported_that_staff_somewhat_gave_care_in_a_professional_way_or_the_facility_was_somewhat_clean','patients_who_reported_that_staff_definitely_gave_care_in_a_professional_way_and_the_facility_was_clean'],
['patients_who_gave_the_facility_a_rating_of_0_to_6_on_a_scale_from_0_lowest_to_10_highest','patients_who_gave_the_facility_a_rating_of_7_or_8_on_a_scale_from_0_lowest_to_10_highest','patients_who_gave_the_facility_a_rating_of_9_or_10_on_a_scale_from_0_lowest_to_10_highest']]
names_list=[['not at all', 'somewhat', 'definitely'],
['no','probably', 'yes definitely'],
['not at all', 'somewhat', 'definitely'],
['0-6','7-8','9-10']]
graphTitle_list=["Patient Ratings of Staff Communication about the Procedure", "Patient Ratings of If They Would Recommend the Facility", "Patient Ratings of Professional Care and Facility Cleanliness","Overall Rating of the Facility (scale from 0 lowest to 10 highest)"]


# plot each of 4 measures

for i, o in enumerate(columns):
    columns=o
    names=names_list[i]
    graphTitle=graphTitle_list[i]
    plot_resp_by_state(columns, names, graphTitle)

In [ ]:
plt_df = mergedMUA_df
fig = px.scatter(plt_df, x="rate_of_readmission", y="patients_rating_of_the_facility_linear_mean_score", color="is-MUA?", hover_data=["facility_name", "address", "city", "state"])
fig.update_layout(template="plotly_white")
fig.update_xaxes(title_text="Readmission Rate")
fig.update_yaxes(title_text="Patients' Mean Rating of the Facility")
fig.for_each_trace(
    lambda trace: trace.update(name=trace.name.replace("is-MUA?=", " ")),
)
fig.show()

In [ ]:
plt_df = mergedMUA_df
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='Yes'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='Yes'].patients_rating_of_the_facility_linear_mean_score, 
    mode='markers', 
    name='MUA',
    marker_color='rgba(255, 182, 193, .9)',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state))
fig.add_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='No'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='No'].patients_rating_of_the_facility_linear_mean_score, 
    mode='markers', 
    name='not MUA',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state))
fig.update_layout(template="plotly_white", 
                  xaxis_title= "Readmission Rate", 
                  yaxis_title="Patients' Mean Rating of the Facility")

fig.show()

In [ ]:
fig=make_subplots(rows=2,cols=2)
fig.append_trace(go.Box(
    x=plt_df['is-MUA?'], 
    y = plt_df['patients_rating_of_the_facility_linear_mean_score'],
    boxpoints='all',
    jitter=0.3, 
    width=.2,
    marker=dict(color='#a2bffe', line=dict(width=.5,color='DarkSlateGrey')),
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state,
    showlegend=False), 
    row=1, col=1)
fig.append_trace(go.Box(
    x=plt_df['is-MUA?'], 
    y = plt_df['patients_recommending_the_facility_linear_mean_score'], 
    boxpoints='all',
    jitter=0.3, 
    width=.2,
    marker=dict(color='#0d75f8', line=dict(width=.5,color='DarkSlateGrey')),
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state,
    showlegend=False), 
    row=1, col=2)
fig.append_trace(go.Box(
    x=plt_df['is-MUA?'], 
    y = plt_df['communication_about_your_procedure_linear_mean_score'], 
    boxpoints='all',
    jitter=0.3, 
    width=.2,
    marker=dict(color='#0e87cc', line=dict(width=.5,color='DarkSlateGrey')),
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state,
    showlegend=False), 
    row=2, col=1)
fig.append_trace(go.Box(
    x=plt_df['is-MUA?'], 
    y = plt_df['rate_of_readmission'], 
    boxpoints='all',
    jitter=0.3, 
    width=.2,
    marker=dict(color='#6a79f7', line=dict(width=.5,color='DarkSlateGrey')),
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state,
    showlegend=False), 
    row=2, col=2)

fig.update_layout(title_text="Outcomes Measures in MUA and non-MUA Facilities",
                  template="plotly_white", 
                 autosize=False,
                 width=900,
                 height=700)
fig.update_xaxes(title_text="is MUA?")
fig.update_yaxes(title_text="Overall Rating", row=1, col=1)
fig.update_yaxes(title_text="Recommendation", row=1, col=2)
fig.update_yaxes(title_text="Communication", row=2, col=1)
fig.update_yaxes(title_text="Readmission Rate", row=2, col=2)


fig.show()

In [ ]:
fig=make_subplots(rows=3,cols=1)
fig.append_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='Yes'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='Yes'].patients_rating_of_the_facility_linear_mean_score, 
    mode='markers', 
    name='MUA',
    marker_color='#cea2fd',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state), 
    row=1, col=1)
fig.append_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='No'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='No'].patients_rating_of_the_facility_linear_mean_score, 
    mode='markers', 
    name='not MUA',
    marker_color='#137e6d',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state), 
    row=1, col=1)
fig.append_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='Yes'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='Yes'].patients_recommending_the_facility_linear_mean_score, 
    mode='markers', 
    marker_color='#cea2fd',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state, 
    showlegend=False), 
    row=2, col=1)
fig.append_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='No'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='No'].patients_recommending_the_facility_linear_mean_score, 
    mode='markers', 
    marker_color='#137e6d',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state, 
    showlegend=False), 
    row=2, col=1)
fig.append_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='Yes'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='Yes'].communication_about_your_procedure_linear_mean_score, 
    mode='markers', 
    marker_color='#cea2fd',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state, 
    showlegend=False), 
    row=3, col=1)
fig.append_trace(go.Scatter(
    x=plt_df[plt_df['is-MUA?']=='No'].rate_of_readmission, 
    y = plt_df[plt_df['is-MUA?']=='No'].communication_about_your_procedure_linear_mean_score, 
    mode='markers', 
    marker_color='#137e6d',
    hoverinfo="text", 
    hovertext=plt_df.facility_name + ' , ' + plt_df.city + ' , ' + plt_df.state, 
    showlegend=False), 
    row=3, col=1)


fig.update_layout(template="plotly_white", 
                 autosize=False,
                 width=900,
                 height=1000)
fig.update_xaxes(title_text="Readmission Rate", row=3, col=1)
fig.update_yaxes(title_text="Overall Rating", row=1, col=1)
fig.update_yaxes(title_text="Recommendation", row=2, col=1)
fig.update_yaxes(title_text="Communication", row=3, col=1)


fig.show()

In [ ]:
columns=[['patients_who_reported_that_staff_did_not_communicate_about_what_to_expect_during_and_after_the_procedure','patients_who_reported_that_staff_somewhat_communicated_about_what_to_expect_during_and_after_the_procedure', 'patients_who_reported_that_staff_definitely_communicated_about_what_to_expect_during_and_after_the_procedure'],
['patients_who_reported_no_they_would_not_recommend_the_facility_to_family_or_friends','patients_who_reported_probably_yes_they_would_recommend_the_facility_to_family_or_friends','patients_who_reported_yes_they_would_definitely_recommend_the_facility_to_family_or_friends'],
['patients_who_reported_that_staff_did_not_give_care_in_a_professional_way_or_the_facility_was_not_clean','patients_who_reported_that_staff_somewhat_gave_care_in_a_professional_way_or_the_facility_was_somewhat_clean','patients_who_reported_that_staff_definitely_gave_care_in_a_professional_way_and_the_facility_was_clean'],
['patients_who_gave_the_facility_a_rating_of_0_to_6_on_a_scale_from_0_lowest_to_10_highest','patients_who_gave_the_facility_a_rating_of_7_or_8_on_a_scale_from_0_lowest_to_10_highest','patients_who_gave_the_facility_a_rating_of_9_or_10_on_a_scale_from_0_lowest_to_10_highest']]
names_list=[['not at all', 'somewhat', 'definitely'],
['no','probably', 'yes definitely'],
['not at all', 'somewhat', 'definitely'],
['0-6','7-8','9-10']]
result=pd.DataFrame(columns=['state','rating','proportion of patients'])

for i, o in enumerate(columns):
    columns=o
    names=names_list[i]
    result = result.merge(resp_by_state(columns, names), how='outer')

result

In [ ]:
fig = px.scatter_geo(location_df, lat='lat', lon='lon', 
                     size="patients_rating_of_the_facility_linear_mean_score", size_max=10,
                     color="is-MUA?",
                     hover_data=["facility_name", "address", "city"],
                     scope='usa')
fig.show()